In [55]:
import torch
from torch.autograd import Variable as V
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import os
from PIL import Image
import numpy as np
print(torch.__version__)

1.3.0


In [56]:
import numpy as np
import json
datanumber = 2

In [57]:
with open ('dataset_ver2/combine.json','r',encoding="utf-8") as data:
    test=json.load(data)
    train_data = []
    label_data = []
    for i in range(datanumber):
        print("-----------------")
        print("location: train",test[i]["location"])
        print("fans_number:",test[i]["fans_number"])
        print("likes:",test[i]["likes"])
        print("comment_number",test[i]["comment_number"])
        train_data.append([test[i]["location"]])
        print(int(test[i]["likes"])/int(test[i]["fans_number"]))
        if int(test[i]["likes"])/(test[i]["fans_number"]/1000) >29:
            a=np.eye(40,k=29)[0]
        else:
            a=np.eye(40,k=int(int(test[i]["likes"])/(int(test[i]["fans_number"])/1000))[0])
        if int(test[i]["comment_number"])/50 >9:
            b=np.eye(40,k=39)[0]
        else:
            b=np.eye(40,k=(int(int(test[i]["comment_number"])/50)+30))[0]
        train_data.append(a+b)
        print((a+b).shape)

-----------------
location: train train/0.jpg
fans_number: 13600
likes: 611
comment_number 7
0.0449264705882353
(40,)
-----------------
location: train train/1.jpg
fans_number: 13600
likes: 587
comment_number 2
0.04316176470588235
(40,)


In [51]:
train_data

[['/test/1.jpg'],
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]),
 ['/test/2.jpg'],
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.])]

In [52]:
train_data_array = np.array(train_data)
train_data_array = train_data_array.reshape(datanumber,2)
print(train_data_array)

[[list(['/test/1.jpg'])
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])]
 [list(['/test/2.jpg'])
  array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])]]


In [53]:
def onehot(length, num):
    d = np.zeros(length)
    d[num] = 1
    return d

In [22]:
# load the image transformer
centre_crop = trn.Compose([
        trn.Resize((256,256)),
        trn.CenterCrop(224),
        trn.ToTensor(),
        trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
class SIGNSDataset(Dataset):
    def __init__(self, data_dir, transform):
        self.filenames = os.listdir(data_dir)
        self.filenames = [os.path.join(data_dir, f) for f in self.filenames if f.endswith('.jpg')]

        #self.labels = [int(os.path.split(filename)[-1][0]) for filename in self.filenames]
        self.labels2 = [int(os.path.split(filename)[-1][0]) for filename in self.filenames]
        #print(self.labels2)
        self.labels = np.array([onehot(6,i) for i in self.labels2])
        #self.filenames = data_dir[:,0]
        #self.labels = data_dir[:,1]
        self.transform = transform

    def __len__(self):
        # return size of dataset
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open(self.filenames[idx])  # PIL image
        #image = Image.open(self.filenames[idx]).convert('LA')  # PIL image
        #image = ImageOps.equalize(image, mask = None)
        image = self.transform(image)
        return image,self.labels[idx]

In [47]:
# th architecture to use
arch = 'resnet18'

# load the pre-trained weights
model_file = '%s_places365.pth.tar' % arch
if not os.access(model_file, os.W_OK):
    weight_url = 'http://places2.csail.mit.edu/models_places365/' + model_file
    os.system('wget ' + weight_url)

In [20]:
model = models.__dict__[arch](num_classes=365)
checkpoint = torch.load(model_file, map_location=lambda storage, loc: storage)
state_dict = {str.replace(k,'module.',''): v for k,v in checkpoint['state_dict'].items()}
model.load_state_dict(state_dict)
model.eval()
#print(model)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [21]:
# for i in model.children():
#     i.requires_grad = False
    
# model.fc =  torch.nn.Sequential(
#             torch.nn.Linear(in_features=512, out_features=128, bias=True),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(in_features=128, out_features=64, bias=True))

#print(model)

In [23]:
# load the class label
file_name = 'categories_places365.txt'
if not os.access(file_name, os.W_OK):
    synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
    os.system('wget ' + synset_url)
classes = list()
with open(file_name) as class_file:
    for line in class_file:
        classes.append(line.strip().split(' ')[0][3:])
classes = tuple(classes)

#print(np.array(classes))

In [24]:
# load the test image
img_name = '12.jpg'
if not os.access(img_name, os.W_OK):
    img_url = 'http://places.csail.mit.edu/demo/' + img_name
    os.system('wget ' + img_url)

img = Image.open(img_name)
input_img = V(centre_crop(img).unsqueeze(0))

# forward pass
logit = model.forward(input_img)

#print(logit)
h_x = F.softmax(logit, 1).data.squeeze()

probs, idx = h_x.sort(0, True)

In [28]:
print('{} prediction on {}'.format(arch,img_name))
# output the prediction
for i in range(0, 5):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))
    print(idx[i])


resnet18 prediction on 12.jpg
0.621 -> patio
tensor(259)
0.296 -> restaurant_patio
tensor(286)
0.021 -> porch
tensor(272)
0.018 -> beer_garden
tensor(53)
0.012 -> courtyard
tensor(109)


In [26]:
model.cpu().eval()
example = torch.rand(1, 3, 256, 256)
traced_script_module = torch.jit.trace(model, example)
traced_script_module.save("model_scope.pt")